In [1]:
!pip install pymongo
!pip install pycld2

In [2]:
from __future__ import print_function

import string
import re
import sys

import pycld2 as cld2

from multiprocessing.pool import Pool
from tqdm._tqdm_notebook import tqdm

from IPython.display import display, clear_output

import numpy as np
import pandas as pd

from os import getcwd, pardir
from os.path import join, basename

from utils.literature import DataLoader, get_document_title, is_english, get_section, get_sections
from utils.preprocessing import NLPPipeline, Tokenizer, Stemmer, ToLowercase, Lemmatizer, StopwordRemover, CitationRemover, SymbolRemover, ContentInBracketsRemover
from glob import glob

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer #TODO: Try lemmatizer instead of stemmer

import pymongo
from pymongo import MongoClient as DBClient

from collections import Counter

nltk.download()

## Specification of the document paths

- download from: [Kaggle](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge)
- extract folder content to `{ProjectDir}/dataset`

In [3]:
root_dir = join(pardir, 'dataset')

json_paths = [
    join(root_dir, 'arxiv', 'arxiv', 'pdf_json'),
    join(root_dir, 'arxiv', 'arxiv', 'pdf_json'),
    join(root_dir, 'comm_use_subset', 'comm_use_subset', 'pdf_json'),
    join(root_dir, 'noncomm_use_subset', 'noncomm_use_subset', 'pdf_json'),
    join(root_dir, 'custom_license', 'custom_license', 'pdf_json'),
    join(root_dir, 'biorxiv_medrxiv', 'biorxiv_medrxiv', 'pdf_json'),
]

files = []
[files.extend(glob(join(path, '*.json'))) for path in json_paths];

## Building the document index

Each document gets stored in the document_index collection with the following layout: 
```json
{
    _id: ..., 
    document_title: ...
}
```

## Building the inverted index

```json
{
    'stem': ...,
    'doc_ids': [
        {
            {'doc_id': ..., 
             'count': {
                 'title': ...,
                 'abstract': ...,
                 'body_text': ...}
             },
             {'doc_id': ..., 
             'count': {
                 'title': ...,
                 'abstract': ...,
                 'body_text': ...}
             }
        }
    ]
}
```

In [4]:
with DBClient('localhost', 27017, w=0) as client:
    db = client['covid_19']
    inverted_index_collection = db['inverted_index']
    inverted_index_collection.create_index([("doc_ids.doc_id", pymongo.ASCENDING)], background=True, name='doc_ids.doc_id')

In [5]:
pipeline = NLPPipeline([
    ToLowercase(),
    CitationRemover(),
    ContentInBracketsRemover(),
    Tokenizer(),
    SymbolRemover(),
    StopwordRemover(),
    Stemmer()
])

In [6]:
def create_post(stem, doc_id, count_object):
    return {'_id': stem, 'doc_ids': [{'doc_id': doc_id, 'count': count_object}]}

In [16]:
def create_count_object(section, count=1):
    if section is 'title': return {'title': count, 'abstract': 0, 'body_text': 0}
    elif section is 'abstract': return {'title': 0, 'abstract': count, 'body_text': 0}
    elif section is 'body_text': return {'title': 0, 'abstract': 0, 'body_text': count}
    else: raise Exception('Invalid section')

In [8]:
def update_document_index(doc_id, document_title, document_index_collection):
    doc_id_entry = document_index_collection.find({'_id': doc_id}).limit(1)
    if doc_id_entry.count() == 0:
        document_index_collection.insert_one({'_id': doc_id, 'document_title': document_title})

In [9]:
def update_inverted_index(doc_id, stemmed_tokens, section, inverted_index_collection):
    #update inverted index
    for stem, count in stemmed_tokens.items():
        stem_entry = inverted_index_collection.find({'_id': stem}).limit(1)
        if stem_entry.count() > 0:
            #try to find current document id in stem entry
            doc_id_object = inverted_index_collection.find(
                {'_id': stem, 
                'doc_ids': {'$elemMatch': {'doc_id': doc_id}}}).limit(1)
            if doc_id_object.count() > 0:
                # update occurrence of stem in document section
                inverted_index_collection.update(
                    {'_id': stem, 
                    'doc_ids': {'$elemMatch': {'doc_id': doc_id}}},
                    {'$inc': {f'doc_ids.$.count.{section}': count}})
            else:
                # add document id
                inverted_index_collection.update(
                    {'_id': stem},
                    {'$push': {'doc_ids': {'doc_id': doc_id, 'count': create_count_object(section, count)}}})
        else:
            post = create_post(stem, doc_id, create_count_object(section, count))
            inverted_index_collection.insert_one(post)

In [10]:
def process_chunk(args, update_doc_idx=True, update_inv_indx=True):
    with DBClient('localhost', 27017, w=0) as client:

        #open a connection to the database
        db = client['covid_19']
        document_index_collection = db['document_index']
        inverted_index_collection = db['inverted_index']
        
        #iterate over all documents in chunk
        for fpath, doc_id in args:
            doc_id = int(doc_id)
            data_loader = DataLoader(fpath)

            doc_title = get_document_title(fpath, data_loader)

            if doc_title == '' or not is_english(doc_title):
                continue

            #database should only contain english documents with an valid document title
            if update_doc_idx:
                update_document_index(
                    doc_id, 
                    doc_title, 
                    document_index_collection)

            if not update_inverted_index:
                continue
            
            for section in get_sections():
                text = get_section(fpath, section, dl=data_loader)
                stemmed_tokens = pipeline.transform(text)
                stemmed_tokens_reduced = Counter(stemmed_tokens)
                update_inverted_index(doc_id, stemmed_tokens_reduced, section, inverted_index_collection)

In [11]:
def create_chunks(files, chunk_size=128):
    doc_ids = list(range(len(files)))
    chunks = list()
    for i in range(0, len(files), chunk_size):
        indices = np.array(doc_ids[i: min(i+chunk_size, len(files))])
        chunks.append(list(zip(
            files[i: min(i+chunk_size, len(files))], 
            doc_ids[i: min(i+chunk_size, len(files))])))
    return chunks

In [12]:
pool = Pool()
chunks = create_chunks(files)

for _ in tqdm(pool.imap_unordered(process_chunk, chunks), total=len(chunks)):
    pass

pool.close()
pool.join()

ome/tobias/anaconda3/lib/python3.7/site-packages/pymongo/network.py", line 255, in _receive_data_on_socket
    chunk_length = sock.recv_into(mv[bytes_read:])
  File "/home/tobias/anaconda3/lib/python3.7/site-packages/pymongo/network.py", line 255, in _receive_data_on_socket
    chunk_length = sock.recv_into(mv[bytes_read:])
  File "/home/tobias/anaconda3/lib/python3.7/site-packages/pymongo/cursor.py", line 787, in count
    cmd, self.__collation, session=self.__session)
  File "/home/tobias/anaconda3/lib/python3.7/site-packages/pymongo/mongo_client.py", line 1464, in _retryable_read
    return func(session, server, sock_info, slave_ok)
  File "/home/tobias/anaconda3/lib/python3.7/site-packages/pymongo/pool.py", line 618, in command
    self._raise_connection_failure(error)
KeyboardInterrupt
  File "/home/tobias/anaconda3/lib/python3.7/site-packages/pymongo/collection.py", line 1600, in _count
    _cmd, self._read_preference_for(session), session)
KeyboardInterrupt
  File "/home/tobias/

KeyboardInterrupt: 

## Performance monitoring

```py
%%timeit -n 100
with DBClient('localhost', 27017, w=0) as client:
    #open a connection to the database
    db = client['covid_19']
    document_index_collection = db['document_index']
    inverted_index_collection = db['inverted_index']
```

785 µs ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
%%timeit -n 100
data_loader = DataLoader(files[0])
doc_title = get_document_title(files[0], data_loader)
```

496 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
data_loader = DataLoader(files[0])
abstract = get_section(files[0], 'abstract', data_loader)

%%timeit -n 100
is_english(abstract)
```

48.8 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
%%timeit -n 100
pipeline.transform(doc_title)
```

1.35 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
%%timeit
inverted_index_collection.find({'_id': 'detect'}).limit(1).count()
```

1.35 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
%%timeit
doc_id_object = inverted_index_collection.find(
                {'_id': 'detect', 
                'doc_ids': {'$elemMatch': {'doc_id': 312}}}).limit(1).count()
```

1.35 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

```py
import uuid

%%timeit
post = create_post(uuid.uuid4(), -1, count_object['abstract'])
inverted_index_collection.insert_one(post)
```

98.7 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

```py
%%timeit
inverted_index_collection.update(
        {'_id': 'testtoken', 
        'doc_ids': {'$elemMatch': {'doc_id': -1}}},
        {'$inc': {f'doc_ids.$.count.abstract': 1}})
```

99.2 µs ± 999 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

```py
%%timeit -n 1000
inverted_index_collection.update(
                    {'_id': 'testtoken'},
                    {'$push': {'doc_ids': {'doc_id': -2, 'count': count_object['abstract']}}})
```

99.2 µs ± 999 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)